In [1]:
import os
import json
import openai
from openai import AzureOpenAI
from datetime import datetime, timedelta
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage, AIMessage, ChatMessage
from datetime import date


In [4]:
    
client = AzureOpenAI(
    api_key =os.getenv("AZURE_OPENAI_API_KEY"),  
    api_version="2024-02-01",
    azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
    )
    
deployment_name='gpt35turbo16k'

In [5]:
today = date.today().strftime("%d-%m-%Y")

In [11]:
tools = [
        {
            "type": "function",
            "function": {
                 "name": "query_db",
                 "description": "Query a database for information on sales, volumes etc",
                 "parameters": {
                    "type": "object",
                    "properties": {
                        "intent" : {
                            "type": "string",
                            "description": "The intent of the query being one of get_sales, get_volume, get_products  "
                        },
                        "country": {
                            "type": "string",
                            "description": "List of countries mentioned as python list"
                        },
                        "start_date": {
                            "type": "string",
                            "description": "The start date for which sales is needed in dd-mm-yyyy format.For example, 01-01-2023",
                        },
                        "end_date": {
                            "type": "string",
                            "description": "The end date for which sales is needed in dd-mm-yyyy format. For example, 31-12-2023",
                        },
                         "products": {
                            "type": "string",
                            "description": "List of products as python list",
                        },
                    },
                "required": ["country"],
             }
            }

        },
             {
            "type": "function",
            "function": {
                 "name": "summarise_doc",
                 "description": "Summarise a page or a document on the internet, given the link",
                 "parameters": {
                    "type": "object",
                    "properties": {
                        "url" : {
                            "type": "string",
                            "description": "The link of the file or page to be summarized"
                        },
                    },
                "required": ["url"],
             }
            }

        }
    ]

In [19]:
user_prompt = f"What is the sales of fries in Germany & IT for last year?"
completion = client.chat.completions.create(
    model=deployment_name,
    messages= [
      {
      "role": "system",
      "content": "1. Consider current date to be " + today +" when extracting start_date and end_date."
      }
      ,
    {
      "role": "user",
      "content": user_prompt
    }],
    max_tokens=100,
    temperature=0.0,
    frequency_penalty=0,
    presence_penalty=0,
    stop=None,
    stream=False,
    tools=tools,
    tool_choice="auto",  # specify the function call
)

output = completion.choices[0].message
print(output)

if output.tool_calls:
        for tool_call in output.tool_calls:
                function_args = json.loads(tool_call.function.arguments)
                print(f"Function arguments: {function_args}")  
                response = execute_query(
                    function_args
                )
            
else:
        print("No tool calls were made by the model.")  


ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_Q7TcQumRgpNBbE3z53tEedyG', function=Function(arguments='{\n  "intent": "get_sales",\n  "country": ["Germany", "Italy"],\n  "start_date": "01-01-2023",\n  "end_date": "31-12-2023",\n  "products": ["fries"]\n}', name='query_db'), type='function')])
Function arguments: {'intent': 'get_sales', 'country': ['Germany', 'Italy'], 'start_date': '01-01-2023', 'end_date': '31-12-2023', 'products': ['fries']}


In [24]:
user_prompt = "Can you summarise this page?"
completion = client.chat.completions.create(
    model=deployment_name,
    messages= [
      {
      "role": "system",
      "content": "1. Consider current date to be " + today +" when extracting start_date and end_date."
      }
      ,
    {
      "role": "user",
      "content": user_prompt
    }],
    max_tokens=100,
    temperature=0.0,
    frequency_penalty=0,
    presence_penalty=0,
    stop=None,
    stream=False,
    tools=tools,
    tool_choice="auto",
)

output = completion.choices[0].message
print(output)

if output.tool_calls:
        for tool_call in output.tool_calls:
                function_args = json.loads(tool_call.function.arguments)
                print(f"Function arguments: {function_args}")  
                response = execute_query(
                    function_args
                )
            
else:
        print("No tool calls were made by the model.")  


ChatCompletionMessage(content='Sure, please provide me with the link to the page you would like me to summarize.', role='assistant', function_call=None, tool_calls=None)
No tool calls were made by the model.


In [21]:
link = "https://sjce.ac.in/wp-content/uploads/2021/10/jnu-Supply-Chain-Management.pdf"

In [23]:
user_prompt = f"Can you summarise this page {link}?"

completion = client.chat.completions.create(
    model=deployment_name,
    messages= [
      {
      "role": "system",
      "content": "Consider current date to be " + today +" when extracting start_date and end_date."
      }
      ,
    {
      "role": "user",
      "content": user_prompt
    }],
    max_tokens=100,
    temperature=0.0,
    frequency_penalty=0,
    presence_penalty=0,
    stop=None,
    stream=False,
    tools=tools,
    tool_choice="auto",  
)

output = completion.choices[0].message
print(output)

if output.tool_calls:
        for tool_call in output.tool_calls:
                function_args = json.loads(tool_call.function.arguments)
                print(f"Function arguments: {function_args}")  
                response = execute_query(
                    function_args
                )
            
else:
        print("No tool calls were made by the model.")  


ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_rXIVUvQwmkMXU3B3nc1jHavI', function=Function(arguments='{\n"url": "https://sjce.ac.in/wp-content/uploads/2021/10/jnu-Supply-Chain-Management.pdf"\n}', name='summarise_doc'), type='function')])
Function arguments: {'url': 'https://sjce.ac.in/wp-content/uploads/2021/10/jnu-Supply-Chain-Management.pdf'}


In [ ]:
## Definition TBD
def execute_query(d):
    ## this could be a call to another LLM to query or API
    
    return 0

In [ ]:
## Definition TBD
def summarise_doc():
    return ""